# Carregar dados
* [download](https://storage.googleapis.com/tensorflow/tf-keras-datasets/minist.npz)
* Mova o arquivo "mnist.npz" para a pasta do projeto
* execute a celula abaixo

obs: é possível carregar o dataset direto pelo keras da seguinte forma:

`tf.keras.datasets.mnist.load_data(path="mnist.npz")`
    
Contudo para evitarmos complicações em lidar com o proxy da Bosch, iremos realizar o download manual do dataset.

In [1]:
import os

from matplotlib import pyplot as plt
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Progbar
import numpy as np

### Parâmetros

Como estaremos trabalhando com o MNIST iremos trabalhar com sua resolução de 28x28x1.
A entrada do gerador será um vetor de valores aleatórios e conforme variamos esses valores diferentes imagens serão geradas. Esse vetor é conhecido como vetor latente e iremos trabalhar com um vetor de tamanho 100

In [2]:
IMG_SHAPE = (28, 28, 1)
LATENT_DIM = 100

opt = Adam(0.0002, 0.5)  # otimizador utilizando tanto pro gerador quanto discriminador


### Generator
Nosso gerador irá ter uma vetor de 100 posições de entrada e a partir desse vetor iremos trabalhar com o reshape pra transformar um vetor em uma matriz e com essa matriz iremos usar upscalings até atingir nossa imagem de saída com dimensão de 28x28x1

In [3]:
generator = Sequential()

generator.add(Input(shape=(LATENT_DIM,)))
generator.add(Dense(128 * 7 * 7, activation="relu"))
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(128, kernel_size=3, padding="same"))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Activation("relu"))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=3, padding="same"))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Activation("relu"))
generator.add(Conv2D(IMG_SHAPE[2], kernel_size=3, padding="same"))
generator.add(Activation("tanh"))

generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6272)              633472    
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 128)       512       
_________________________________________________________________
activation (Activation)      (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 128)       0

### Discriminator
O papel do nosso discriminador será de classificar as imagens de entrada e dizer se elas são reais ou criadas pelo nosso generator. Para isso teremos um problema de classificação binária e utilizaremos uma sigmoid e o "binary_crossentropy" como função custo.

In [4]:
discriminator = Sequential()

discriminator.add(Input(shape=IMG_SHAPE))
discriminator.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))
discriminator.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
discriminator.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))
discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))
discriminator.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.summary()

discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(0.0001, 0.5),
                      metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 64)         

### Modelo combinado

In [5]:
discriminator.trainable = False  # Não queremos atualizar os pesos do discriminador durante o treino do gerador
combined = Model(generator.input, discriminator(generator.output))
combined.compile(loss='binary_crossentropy', optimizer=opt)

KerasTensor(type_spec=TensorSpec(shape=(None, 100), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


### Dataset
Nesse caso iremos seguir as intruções citadas no artigos original sobre qual tipo de pre processamento executar em nosso dados.
Vamos normalizar nosso dataset com valores entre -1 e 1

In [6]:
with np.load(r"mnist.npz", allow_pickle=True) as f:
    x_train = f['x_train']

x_train = x_train / 127.5 - 1.
x_train = np.expand_dims(x_train, axis=3)

### Treinamento

O treinamento de um GAN é algo que fode dos padrões da função .fit() do keras, por conta de estarmos treinando duas redes simultaneamente, por isso vamos criar nosso propio pipeline de treinamento.

In [7]:
# hyperparameters
batch_size = 32
epochs = 4000
steps_per_epoch = 50
metrics_names = ['d_loss','acc', 'g_loss'] 

In [14]:
# Adversarial ground truths
valid = np.ones((batch_size, 1))  # todas iamgens reais terão label como 1 (verdadeiro)
fake = np.zeros((batch_size, 1))  # todas imagens falsas teraão label como 0 (falso)
os.makedirs("./dcgan", exist_ok=True)  # pasta para salvar as imagens monstrando o progresso do nosso treinamento

for epoch in range(epochs):
    print(f"\nepoch {epoch + 1}/{epochs}")

    pb_i = Progbar(steps_per_epoch, stateful_metrics=metrics_names)  # Progbar para termos um feedback igual a função .fit
    for step in range(steps_per_epoch):
        idx = np.random.randint(0, x_train.shape[0], batch_size)  # seleção aleatória de imagens reais
        imgs = x_train[idx]

        # geração de imagens fake
        noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
        gen_imgs = generator.predict(noise)

        # Treinamento do discriminador
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


        # Treinamento do generator, utlizamos os labels de verdareiro para nossos fake pois afinal queremos enganar nosso 
        # discriminador
        g_loss = combined.train_on_batch(noise, valid)

        # Atualiza nosso progess bar
        values=[('acc', d_loss[1]), ('d_loss', d_loss[0]), ('g_loss', g_loss)]
        pb_i.add(1, values=values)
        
    # Nosso "callback" on_epoch_end
    # Salva uma imagem com 25 caracteres gerados pelo nosso gerador.
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, LATENT_DIM))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1  # como usamos o matplotlib, não há necessidade de deixar na escala de 0-255 como o opencv
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"./dcgan/mnist_{epoch}.png")
    plt.close()


epoch 1/4000
29/50 [================>.............] - ETA: 10s - acc: 1.0000 - d_loss: 2.7467e-05 - g_loss: 1.0781

KeyboardInterrupt: 